# **UTS**

## **Analisa cluster terhadap data abstrak prodi teknik informatika pta.trunojoyo.ac.id dengan menggunakan model k-mean clustering dan preprosessing (k=2_ dan menggunakan  reduksi dimensi PCA (principal component analysis)**

In [1]:
import numpy as np
import pandas as pd

mengimport data dari file csv yang sudah di upload di github

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/AmandaCaecilia/prosainsdata/main/dataset-baru.csv")
df.drop(columns=['Unnamed: 0', 'Kategori'])

,Abstrak
0,Sistem informasi akademik (SIAKAD) merupaka...
1,Berjalannya koneksi jaringan komputer dengan l...
2,Web server adalah sebuah perangkat lunak serve...
3,Penjadwalan kuliah di Perguruan Tinggi me...
4,Seiring perkembangan teknologi yang ada diduni...
...,...
800,Investasi saham selama ini memiliki resiko ker...
801,Information Retrieval (IR) merupakan pengambil...
802,Klasifikasi citra merupakan proses pengelompok...
803,Identifikasi atribut pejalan kaki merupakan sa...


### **Normalisasi teks**

In [3]:
!pip install nltk
!pip install sastrawi
!pip install swifter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 KB 7.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 830.9/830.9 KB 18.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 62.1 MB/s eta 0:00:00
  Created wheel for swifter: filename=swifter-1.3.4-py3-none-any.whl size=16321 sha256=6bb4f7389927850f483b67ed678dfa16025a611b8216aa0812b77c0c4559fb12
  Stored in directory: /root/.cache/pip/wheels/2b/5e/f2/3931524f702ffd03309e96d35ee2fbf9c61c27377511ee8d4c
Successfully built swifter


In [4]:
import nltk
from nltk.corpus import stopwords
import re
import swifter

In [5]:
text = df['Abstrak']
text

0      Sistem  informasi  akademik  (SIAKAD) merupaka...
1      Berjalannya koneksi jaringan komputer dengan l...
2      Web server adalah sebuah perangkat lunak serve...
3      Penjadwalan  kuliah  di  Perguruan  Tinggi  me...
4      Seiring perkembangan teknologi yang ada diduni...
                             ...                        
800    Investasi saham selama ini memiliki resiko ker...
801    Information Retrieval (IR) merupakan pengambil...
802    Klasifikasi citra merupakan proses pengelompok...
803    Identifikasi atribut pejalan kaki merupakan sa...
804    Topik deteksi objek telah menarik perhatian ya...
Name: Abstrak, Length: 805, dtype: object

Case folding

In [6]:
def case_folding(comment):
    comment = comment.lower()
    return comment
text = text.apply(case_folding)
text

0      sistem  informasi  akademik  (siakad) merupaka...
1      berjalannya koneksi jaringan komputer dengan l...
2      web server adalah sebuah perangkat lunak serve...
3      penjadwalan  kuliah  di  perguruan  tinggi  me...
4      seiring perkembangan teknologi yang ada diduni...
                             ...                        
800    investasi saham selama ini memiliki resiko ker...
801    information retrieval (ir) merupakan pengambil...
802    klasifikasi citra merupakan proses pengelompok...
803    identifikasi atribut pejalan kaki merupakan sa...
804    topik deteksi objek telah menarik perhatian ya...
Name: Abstrak, Length: 805, dtype: object

Menghilangkan url

In [7]:
def linkNormalize(text):
    text = re.sub(r"\s—\s", "", text)
    text = re.sub(r"http\S+", "", text)
    return text
text = text.apply(linkNormalize)
text

0      sistem  informasi  akademik  (siakad) merupaka...
1      berjalannya koneksi jaringan komputer dengan l...
2      web server adalah sebuah perangkat lunak serve...
3      penjadwalan  kuliah  di  perguruan  tinggi  me...
4      seiring perkembangan teknologi yang ada diduni...
                             ...                        
800    investasi saham selama ini memiliki resiko ker...
801    information retrieval (ir) merupakan pengambil...
802    klasifikasi citra merupakan proses pengelompok...
803    identifikasi atribut pejalan kaki merupakan sa...
804    topik deteksi objek telah menarik perhatian ya...
Name: Abstrak, Length: 805, dtype: object

Menghilangkan karakter tertentu

In [8]:
symbols = ",!\"#$%&()*+-.…/:;<=>?@[\]^_`{|}~\n"
def cleaning_text(comment):
    for i in symbols:
        comment = np.char.replace(comment, i, ' ')
    return comment
text = text.apply(cleaning_text)
text

0      sistem  informasi  akademik   siakad  merupaka...
1      berjalannya koneksi jaringan komputer dengan l...
2      web server adalah sebuah perangkat lunak serve...
3      penjadwalan  kuliah  di  perguruan  tinggi  me...
4      seiring perkembangan teknologi yang ada diduni...
                             ...                        
800    investasi saham selama ini memiliki resiko ker...
801    information retrieval  ir  merupakan pengambil...
802    klasifikasi citra merupakan proses pengelompok...
803    identifikasi atribut pejalan kaki merupakan sa...
804    topik deteksi objek telah menarik perhatian ya...
Name: Abstrak, Length: 805, dtype: object

Menghilangkan Stopwords menggunakan library nltk

In [9]:
nltk.download('stopwords')
list_stopwords = stopwords.words('indonesian')
def stopword_removal(comment):
    comment = comment.split()
    data = [word for word in comment if word not in list_stopwords]
    kalimat = ""
    for i in data:
      kalimat += i
      kalimat += " "
    return kalimat

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Memperbaiki kata yang yang tidak baku menggunakan library sastrawi

In [10]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()
def stemmingNormalize(comment):
    comment = comment.split()
    data = []
    for term in comment:
        term = stemmer.stem(term)
        data.append(term)
    kalimat = ""
    for i in data:
      kalimat += i
      kalimat += " "
    return kalimat
text = text.swifter.apply(stemmingNormalize)
text

Pandas Apply:   0%|          | 0/805 [00:00<?, ?it/s]

0      sistem informasi akademik siakad rupa sistem i...
1      jalan koneksi jaring komputer dengan lancar da...
2      web server adalah buah perangkat lunak server ...
3      jadwal kuliah di guru tinggi rupa masalah yang...
4      iring kembang teknologi yang ada dunia muncul ...
                             ...                        
800    investasi saham lama ini milik resiko rugi yan...
801    information retrieval ir rupa ambil informasi ...
802    klasifikasi citra rupa proses kelompok piksel ...
803    identifikasi atribut pejal kaki rupa salah sat...
804    topik deteksi objek telah tarik perhati yang b...
Name: Abstrak, Length: 805, dtype: object

### **Perhitungan menggunakan TF-IDF scikit learn**

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_separate = tfidf_vectorizer.fit_transform(text)

df_tfidf = pd.DataFrame(
    tfidf_separate.toarray(), columns=tfidf_vectorizer.get_feature_names_out(), index=df.index
)
X = df_tfidf.values
df_tfidf

,00,000,00001,0001,000505341,000506436,000792353,00086,00089,001,...,zara,zat,zcz,zf,zona,zone,zoning,zoom,zucara,zungu
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
801,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
802,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
803,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


PCA

In [15]:
from sklearn.decomposition import PCA

pca = PCA(n_components =10)

X = pca.fit_transform(X)
# X_test = pca.transform(X_test)

pca.explained_variance_ratio_

array([0.02031107, 0.01607337, 0.0131176 , 0.01121444, 0.00903131,
       0.00841368, 0.00781091, 0.00732616, 0.0071461 , 0.00671337])

K-Means

In [16]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(X)

KMeans(n_clusters=2, random_state=42)

In [17]:
center = kmeans.cluster_centers_
center

array([[ 0.30814794,  0.08342842,  0.00897638,  0.0545473 , -0.00671188,
        -0.00639137,  0.01274177,  0.02285693, -0.00319882,  0.00829345],
       [-0.04420872, -0.01196913, -0.0012878 , -0.00782568,  0.00096293,
         0.00091694, -0.00182801, -0.00327919,  0.00045892, -0.00118983]])

In [18]:
y_pred = kmeans.labels_


data = kmeans.labels_
res = []
for i in data:
    if i == 0:
        res.append("RPL")
    else:
        res.append("KOMPUTASI")
print(res)

['KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'RPL', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'RPL', 'KOMPUTASI', 'RPL', 'KOMPUTASI', 'RPL', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'RPL', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'RPL', 'KOMPUTASI', 'RPL', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'RPL', 'RPL', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'RPL', 'KOMPUTASI', 'KOMPUTASI', 'RPL', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'RPL', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMP

In [19]:
y_true = df['Kategori'].values.tolist()
print(df['Kategori'].values.tolist())

['RPL', 'RPL', 'RPL', 'KOMPUTASI', 'RPL', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'RPL', 'RPL', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'RPL', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'RPL', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'RPL', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'RPL', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'RPL', 'RPL', 'KOMPUTASI', 'RPL', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'RPL', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'RPL', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'RPL', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'RPL', 'KOMPUTASI', 'RPL', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'K

In [20]:
n = len(y_true)
for i in range(n):
    print(res[i],"\t : ",y_true[i])

KOMPUTASI 	 :  RPL
KOMPUTASI 	 :  RPL
KOMPUTASI 	 :  RPL
KOMPUTASI 	 :  KOMPUTASI
KOMPUTASI 	 :  RPL
KOMPUTASI 	 :  KOMPUTASI
KOMPUTASI 	 :  KOMPUTASI
RPL 	 :  KOMPUTASI
KOMPUTASI 	 :  KOMPUTASI
KOMPUTASI 	 :  RPL
KOMPUTASI 	 :  RPL
KOMPUTASI 	 :  KOMPUTASI
KOMPUTASI 	 :  KOMPUTASI
KOMPUTASI 	 :  KOMPUTASI
KOMPUTASI 	 :  KOMPUTASI
KOMPUTASI 	 :  KOMPUTASI
RPL 	 :  KOMPUTASI
KOMPUTASI 	 :  KOMPUTASI
RPL 	 :  KOMPUTASI
KOMPUTASI 	 :  KOMPUTASI
RPL 	 :  KOMPUTASI
KOMPUTASI 	 :  KOMPUTASI
KOMPUTASI 	 :  KOMPUTASI
KOMPUTASI 	 :  KOMPUTASI
KOMPUTASI 	 :  KOMPUTASI
KOMPUTASI 	 :  KOMPUTASI
RPL 	 :  KOMPUTASI
KOMPUTASI 	 :  KOMPUTASI
KOMPUTASI 	 :  KOMPUTASI
KOMPUTASI 	 :  RPL
KOMPUTASI 	 :  KOMPUTASI
KOMPUTASI 	 :  KOMPUTASI
KOMPUTASI 	 :  KOMPUTASI
KOMPUTASI 	 :  RPL
KOMPUTASI 	 :  KOMPUTASI
RPL 	 :  KOMPUTASI
KOMPUTASI 	 :  KOMPUTASI
RPL 	 :  KOMPUTASI
KOMPUTASI 	 :  KOMPUTASI
KOMPUTASI 	 :  RPL
KOMPUTASI 	 :  KOMPUTASI
KOMPUTASI 	 :  KOMPUTASI
KOMPUTASI 	 :  KOMPUTASI
KOMPUTASI 	 :  KOMPUT

In [21]:
from sklearn.metrics import accuracy_score
print ("Accuracy : ",
    accuracy_score(y_true, res)*100)

Accuracy :  53.7888198757764
